# Lesiones Personales, Pirateria & Secuestro

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [26]:
PATH_FILE = '/content/drive/MyDrive/Police-Data/Bronce'
OUTPUT_PATH_FILE = '/content/drive/MyDrive/Police-Data/Silver'
CODIGO_DANE_PATH = '/content/drive/MyDrive/Police-Data/dane_codes.xlsx'

## Libraries

In [27]:
import pandas as pd
import numpy as np
import os
import glob

In [28]:
def find_header_row(file_path):
    """
    Finds the header line with a column name
    """
    column_name = ['DEPARTAMENTO','DEPARTAMENTO ','Departamento','departamento']
    header_row = None

    df = pd.read_excel(file_path, header=None)

    for i, row in df.iterrows():
        if column_name[0] in row.values or column_name[1] in row.values or column_name[2] in row.values or column_name[3] in row.values:
            header_row = i
            break
        if i>20:
            break

    if header_row is None:
        raise ValueError(f"No se encontró la fila de encabezado en el archivo {file_path}")

    return header_row

def read_excel_with_variable_header(file_path):
    header_row = find_header_row(file_path)

    df = pd.read_excel(file_path, header=header_row)

    total_row_index = df[df.iloc[:, 0].str.contains("TOTAL", case=False, na=False)].index.min()

    if total_row_index is not None:
        df = df.iloc[:total_row_index]

    return df

## Read Files

In [29]:
folders = ["Lesiones_Personales","Pirateria_Terrestre","Secuestro"]

dataframes_by_folder = {}

def folder_read(folders):
  for folder in folders:
      file_paths = glob.glob(os.path.join(PATH_FILE+f'/{folder}/','*.xlsx')) + glob.glob(os.path.join(PATH_FILE+f'/{folder}/','*.xls'))
      folder_dataframes = []
      for file_path in file_paths:
          try:
              df = read_excel_with_variable_header(file_path)
              folder_dataframes.append(df)
          except ValueError as e:
              print(e)
      dataframes_by_folder[folder] = folder_dataframes

In [30]:
folder_read(folders)

In [31]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS','ARMAS MEDIO':'ARMAS_MEDIOS',
        'CANTIDAD ': 'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

## Lesiones Personales

In [32]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Lesiones_Personales']]
lesiones_personales_df = pd.concat(standardized_dataframes, ignore_index=True)

In [33]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['ACIDO','AGUA CALIENTE','ALIMENTOS VENCIDOS','ALUCINOGENOS','ARMA BLANCA / CORTOPUNZANTE','ARMA DE FUEGO','ARMA TRAUMATICA',
                             'ARTEFACTO EXPLOSIVO/CARGA DINAMITA','ARTEFACTO INCENDIARIO','BOLSA PLASTICA','CARRO BOMBA','CINTAS/CINTURON','COMBUSTIBLE',
                             'CONTUNDENTES','ESCOPOLAMINA','GASES','GRANADA DE MANO','JERINGA','LICOR ADULTERADO','MEDICAMENTOS','MINA ANTIPERSONA','MOTO',
                             'PAPA EXPLOSIVA','PERRO','POLVORA(FUEGOS PIROTECNICOS)','PRENDAS DE VESTIR','QUIMICOS','SIN EMPLEO DE ARMAS',
                             'SUSTANCIAS TOXICAS','VEHICULO','VENENO','OLLA BOMBA','PUNZANTES','PAQUETE BOMBA','CUCHILLA','ALMOHADA','CORTANTES','DIRECTA',
                             'ARMAS BLANCAS','CILINDRO BOMBA','CINTURON BOMBA']
lesiones_personales_df["AGRUPA_EDAD_PERSONA"] = lesiones_personales_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
lesiones_personales_df["GENERO"] = lesiones_personales_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
lesiones_personales_df["ARMAS_MEDIOS"] = lesiones_personales_df["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")

In [34]:
dane_codes = pd.read_excel(CODIGO_DANE_PATH,sheet_name='Municipios_dane',dtype={'CODIGO_DANE':str})
def clean_data(data):
  temp = data.copy()
  temp['CODIGO_DANE'] = temp['CODIGO_DANE'].astype(str)
  temp['CODIGO_DANE'] = temp['CODIGO_DANE'].apply(lambda x: '0' + x if len(x) == 9 else x)
  temp['CODIGO_DANE'] = temp['CODIGO_DANE'].str[:5]
  filtered_df = temp[temp['CODIGO_DANE'].apply(lambda x: x[-1].isdigit() if x else False)]
  # Adjust upper case
  filtered_df = filtered_df.applymap(lambda x: x.upper() if isinstance(x, str) else x)
  # Merge DANE CODE
  merged_df = filtered_df.merge(dane_codes, on='CODIGO_DANE', how='left', suffixes=('', '_csv'))
  # fill Municipio
  merged_df['MUNICIPIO'] = merged_df['MUNICIPIO_csv']
  merged_df = merged_df.drop(columns=['MUNICIPIO_csv'])
  merged_df['CAPITAL'] = merged_df['CODIGO_DANE'].apply(lambda codigo: 1 if codigo[-3:] == '001' else 0)
  # drop sufix (1), (2), (3)
  merged_df['MUNICIPIO'] = merged_df['MUNICIPIO'].str.replace(r'\s*\(\d+\)$', '', regex=True)
  # filter date
  output_df = merged_df[(merged_df['FECHA'] >= '2013-01-01') & (merged_df['FECHA'] <= '2023-12-31')]
  return output_df

In [35]:
lesiones_personales_filtered = clean_data(lesiones_personales_df)

In [36]:
lesiones_personales_output = lesiones_personales_filtered.groupby(
    ["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS","CAPITAL"]).sum().reset_index()

In [37]:
lesiones_personales_output.to_csv(OUTPUT_PATH_FILE+"/lesiones_personales.csv",index=False)

## Piratería Terrestre

In [38]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Pirateria_Terrestre']]
pirateria_terrestre_df = pd.concat(standardized_dataframes, ignore_index=True)

In [39]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['ARMA BLANCA / CORTOPUNZANTE','ARMA DE FUEGO','CONTUNDENTES','ESCOPOLAMINA','SIN EMPLEO DE ARMAS',
                             'CORTOPUNZANTES','CORTANTES','PUNZANTES']
pirateria_terrestre_df["AGRUPA_EDAD_PERSONA"] = pirateria_terrestre_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
pirateria_terrestre_df["GENERO"] = pirateria_terrestre_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
pirateria_terrestre_df["ARMAS_MEDIOS"] = pirateria_terrestre_df["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")

In [40]:
pirateria_terrestre_filtered = clean_data(pirateria_terrestre_df)

In [41]:
pirateria_terrestre_output = pirateria_terrestre_filtered.groupby(
    ["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS","CAPITAL"]).sum().reset_index()

In [42]:
pirateria_terrestre_output.to_csv(OUTPUT_PATH_FILE+"/pirateria_terrestre.csv",index=False)

## Secuestro

In [43]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'DELITO', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS','ARMAS MEDIO':'ARMAS_MEDIOS',
        'DESCRIPCION CONDUCTA':'DELITO','DELITO ':'DELITO',
        'CANTIDAD ': 'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

In [44]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Secuestro']]
secuestro_df = pd.concat(standardized_dataframes, ignore_index=True)

In [45]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['ARMA BLANCA / CORTOPUNZANTE','ARMA DE FUEGO','SIN EMPLEO DE ARMAS','ARMAS BLANCAS','CONTUNDENTES','ESPOSAS']
secuestro_df["AGRUPA_EDAD_PERSONA"] = secuestro_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
secuestro_df["GENERO"] = secuestro_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
secuestro_df["ARMAS_MEDIOS"] = secuestro_df["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")
secuestro_df["DELITO"] = secuestro_df["DELITO"].fillna("NO REPORTADO")

In [46]:
secuestro_df['FECHA'] = pd.to_datetime(secuestro_df['FECHA'])
secuestro_df['FECHA'] = secuestro_df['FECHA'].dt.strftime('%Y-%m-%d')

In [47]:
secuestro_filtered = clean_data(secuestro_df)

In [48]:
secuestro_output = secuestro_filtered.groupby(
    ["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS","DELITO","CAPITAL"]).sum().reset_index()

In [49]:
secuestro_output.to_csv(OUTPUT_PATH_FILE+"/secuestro.csv",index=False)